In [1]:
import configparser
import numpy as np
import pandas as pd
import tasks.models as m

from sqlalchemy import create_engine, func, distinct
from sqlalchemy.orm import Session
from sqlalchemy import tuple_
import redis

In [2]:
config = configparser.ConfigParser()
config.read("config/config.ini")

engine = create_engine(config["DB"].get("db_url"))
session = Session(engine)

r = redis.Redis(
    host=config["REDIS"].get("host"),
    port=config["REDIS"].get("port"),
    db=config["REDIS"].get("db"),
)

In [3]:
s = (
    session.query(m.Redditor_Subreddit_Comment.redditor_id)
    .group_by(m.Redditor_Subreddit_Comment.redditor_id)
    .having(func.count(m.Redditor_Subreddit_Comment.subreddit_id.distinct()) > 2)
    .subquery()
)
redditor_subreddit_comment = np.array(
    session.query(
        m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name, m.Redditor.name
    )
    .distinct(m.Redditor_Subreddit_Comment.redditor_id, m.Subreddit.name)
    .filter(m.Redditor_Subreddit_Comment.redditor_id.in_(s))
    .join(m.Subreddit, m.Redditor_Subreddit_Comment.subreddit_id == m.Subreddit.id)
    .join(m.Redditor, m.Redditor_Subreddit_Comment.redditor_id == m.Redditor.id)
    .all()
).T
df = pd.DataFrame(
    {
        "user_id": redditor_subreddit_comment[0],
        "subreddit_id": redditor_subreddit_comment[1],
        "username": redditor_subreddit_comment[2],
    }
)

In [4]:
user_ids = np.unique(df['user_id'])

In [5]:
def compare_recommendation_and_interaction(user_id):
    if type(user_id) != list:
        user_id = [user_id]
    for u in user_id:
        print(f"user: {df[df['user_id']==u]['username'].iloc[0]}") 
        print(f"commented on {np.unique(df[df['user_id']==u]['subreddit_id'])}")
        print(f"recommended {list(map(lambda x: x.decode(), r.lrange(u, 0, -1)))}")
        print("")


In [6]:
compare_recommendation_and_interaction(user_ids[0])

user: arq25
commented on ['StockMarket' 'sysadmin' 'whiskey']
recommended ['pennystocks', 'bourbon', 'RobinHoodPennyStocks', 'options', 'Daytrading']



In [7]:
compare_recommendation_and_interaction(user_ids[1])

user: ac13332
commented on ['AskUK' 'britishproblems' 'golf']
recommended ['CasualUK', 'unitedkingdom', 'ukpolitics', 'london', 'UKPersonalFinance']



In [8]:
compare_recommendation_and_interaction(user_ids[2])

user: ZetTheLegendaryHero
commented on ['DCcomics' 'Doom' 'batman' 'goodanimemes']
recommended ['DC_Cinematic', 'Spiderman', 'SpidermanPS4', 'raimimemes', 'starwarsmemes']



In [9]:
compare_recommendation_and_interaction(user_ids[3])

user: M6D_Magnum
commented on ['MilitaryPorn' 'assassinscreed' 'gundeals' 'guns']
recommended ['ar15', 'GunPorn', 'progun', 'EDC', 'knives']



In [10]:
compare_recommendation_and_interaction(user_ids[4])

user: Eggith
commented on ['askgaybros' 'dbz' 'tf2']
recommended ['gay', 'gaybros', 'gay_irl', 'DBZDokkanBattle', 'gaymers']



In [11]:
compare_recommendation_and_interaction(user_ids[5])

user: Awesomefoxhound
commented on ['eu4' 'reddeadredemption2' 'selfimprovement']
recommended ['hoi4', 'CrusaderKings', 'RedDeadOnline', 'DecidingToBeBetter', 'imaginarymaps']



In [12]:
compare_recommendation_and_interaction(user_ids[10])

user: walnoter
commented on ['animememes' 'dndmemes' 'furry']
recommended ['wholesomeanimemes', 'goodanimemes', 'Konosuba', 'awwnime', 'MemeTemplatesOfficial']



In [13]:
compare_recommendation_and_interaction(user_ids[110])

user: yinyang107
commented on ['RimWorld' 'toronto' 'worldbuilding']
recommended ['ontario', 'PersonalFinanceCanada', 'torontoraptors', 'Stellaris', 'vancouver']



In [14]:
compare_recommendation_and_interaction(user_ids[1000])

user: SawkyScribe
commented on ['BokuNoHeroAcademia' 'SpidermanPS4' 'patientgamers']
recommended ['Spiderman', 'raimimemes', 'Naruto', 'thelastofus', 'ShingekiNoKyojin']



In [16]:
# compare_recommendation_and_interaction('10rndb')
compare_recommendation_and_interaction(list(np.unique(df[df['subreddit_id']=='learnpython']['user_id'])))

user: onequestion1168
commented on ['astrology' 'conspiracytheories' 'learnpython' 'sysadmin']
recommended ['cybersecurity', 'linuxquestions', 'homelab', 'tarot', 'spirituality']

user: xenaprincesswarlord
commented on ['HowToHack' 'TechNewsToday' 'UKPersonalFinance' 'learnpython']
recommended ['datascience', 'cybersecurity', 'linuxquestions', 'AskUK', 'linuxmasterrace']

user: Badger87000
commented on ['EscapefromTarkov' 'TheRightCantMeme' 'learnpython']
recommended ['playrust', 'linuxquestions', 'feedthebeast', 'linuxmasterrace', 'datascience']

user: MuttsNStuff
commented on ['learnpython' 'minimalism' 'seduction']
recommended ['simpleliving', 'dating', 'datingoverthirty', 'webdev', 'linuxquestions']

user: str8nobull
commented on ['Doom' 'EngineeringStudents' 'learnpython']
recommended ['AskEngineers', 'datascience', 'learnmath', 'engineering', 'Ubuntu']

user: SweeTLemonS_TPR
commented on ['Coffee' 'LeopardsAteMyFace' 'Rabbits' 'TheMonkeysPaw'
 'cscareerquestions' 'cybersecurity' 